In [48]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [49]:
T = 1.0
F = 0.0
bias = 1.0

In [50]:
def get_AND_data():
    X = [
            [F, F, bias],
            [F, T, bias],
            [T, F, bias],
            [T, T, bias]
        ]
    Y = [[F], [F], [F], [T]]
    return X, Y

def get_OR_data():
    X = [
            [F, F, bias],
            [F, T, bias],
            [T, F, bias],
            [T, T, bias]
        ]
    Y = [[F], [T], [T], [T]]
    return X, Y

def get_XOR_data():
    X = [
            [F, F, bias],
            [F, T, bias],
            [T, F, bias],
            [T, T, bias]
        ]
    Y = [[F], [T], [T], [F]]
    return X, Y

In [51]:
X, Y = get_AND_data()

In [65]:
class Perceptron:
    def __init__(self):
        # 논리연산을 위한 입력값 X, Y와 편향값 (b)를 받을 것이므로, weight를 [3,1]로 설정합니다.  
        # 3은 세개의 입력을 의미하고, 1은 한개의 뉴론임을 의미합니다.
        self.W = tf.Variable(tf.random.normal([3, 1]))
    
    def train(self,X):
        err = 1
        epoch, max_epochs = 0, 20
        while err > 0.0 and epoch < max_epochs:
            epoch += 1
            self.optimize(X)
            # MSE (평균제곱오차)를 관찰하며, 학습이 진행되는 동안, 에러(MSE)가 줄어듬을 확인합니다.
            err = self.mse(Y, self.pred(X))
            print('epoch:', epoch, 'mse:', err)
    
    def pred(self, X):
        return self.step(tf.matmul(X, self.W))
       
    def mse(self, Y, Y_hat):
        return tf.reduce_mean(tf.square(tf.subtract(Y, Y_hat)))
    
    def step(self, X):
        # step(x) = { 1 if x > 0; 0 otherwise }
        return tf.dtypes.cast(tf.math.greater(X, 0), tf.float32)

    def optimize(self, X):
        """
        퍼셉트론은 경사하강법을 사용한 최적화가 불가능합니다.
        매번 학습을 진행할 때마다 가중치를 아래의 룰에 맞게 업데이트합니다.  

        if target == 1 and activation == 0:  
          w_new = w_old + input  

        if target == 0 and activation == 1:  
          w_new = w_old - input  

        위의 두가지 조건은 아래의 코드로 간단히 구현 가능합니다.  
        """
        delta = tf.matmul(X, tf.subtract(Y, self.step(tf.matmul(X, self.W))), transpose_a=True)
        self.W.assign(self.W+delta)

In [66]:
perceptron = Perceptron()

In [67]:
perceptron.train(X)

epoch: 1 mse: Tensor("Mean_8:0", shape=(), dtype=float32)


TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

In [71]:
class Node:
    def __init__(self):
        self.w = tf.Variable(tf.random.normal([2, 1]))
        self.b = tf.Variable(tf.random.normal([1, 1]))
        
    def __call__(self, x):
        return self.preds(x)
        
    def preds(self, x):
        #순전파(forward propagation)
        out = tf.matmul(x, self.w)
        out = tf.add(out, self.b)
        out = tf.nn.sigmoid(out)
        return out
    
    def loss(self, y_pred, y):
        return tf.reduce_mean(tf.square(y_pred - y))
    
    def train(self, inputs, outputs, learning_rate):
        epochs = range(10000)
        for epoch in epochs:
            with tf.GradientTape() as t:
                current_loss = self.loss(self.preds(inputs), outputs)
                #역전파(back propagation)
                dw, db = t.gradient(current_loss, [self.w, self.b])
                self.w.assign_sub(learning_rate * dw)
                self.b.assign_sub(learning_rate * db)

In [74]:
#And operation
inputs = tf.constant([[0.0,0.0], [0.0,1.0], [1.0,0.0], [1.0,1.0]])
outputs = tf.constant([[0.0], [0.0], [0.0], [1.0]])

node = Node()
#train
node.train(inputs, outputs, 0.01)

#test
assert node([[0.0,0.0]]).numpy()[0][0] < 0.5
assert node([[0.0,1.0]]).numpy()[0][0] < 0.5
assert node([[1.0,0.0]]).numpy()[0][0] < 0.5
assert node([[1.0,1.0]]).numpy()[0][0] >= 0.5

AttributeError: 'RefVariable' object has no attribute '_id'

In [75]:
#And operation
inputs = tf.constant([[0.0,0.0], [0.0,1.0], [1.0,0.0], [1.0,1.0]])
outputs = tf.constant([[0.0], [1.0], [1.0], [1.0]])

node = Node()
#train
node.train(inputs, outputs, 0.01)

#test
assert node([[0.0,0.0]]).numpy()[0][0] < 0.5
assert node([[0.0,1.0]]).numpy()[0][0] >= 0.5
assert node([[1.0,0.0]]).numpy()[0][0] >= 0.5
assert node([[1.0,1.0]]).numpy()[0][0] >= 0.5

AttributeError: 'RefVariable' object has no attribute '_id'

In [76]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import numpy as np

In [78]:
X = np.array([[0.,0.],[0.,1.],[1.,0.],[1.,1.]])
y = np.array([0.,1.,1.,0.])

In [79]:
model = Sequential()

#첫 번째 히든 레이어
model.add(Dense(units=2, activation='sigmoid', input_dim=2))

#두 번째 히든 레이어
model.add(Dense(units=1, activation='sigmoid'))

In [81]:
#손실 함수 및 최적화 정의
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [82]:
#모델 요약
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 6         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
None


In [85]:
#모델 학습, 10분 정도 걸립니다. 모델 학습 진행 상황을 보시려면 verbose=1로 수정해주세요.
model.fit(X,y,epochs=50,batch_size=4,verbose=1)

Epoch 1/50
4/4 [==============================] - 0s 500us/sample - loss: 0.7029 - acc: 0.5000
Epoch 2/50
4/4 [==============================] - 0s 500us/sample - loss: 0.7029 - acc: 0.5000
Epoch 3/50
4/4 [==============================] - 0s 500us/sample - loss: 0.7029 - acc: 0.5000
Epoch 4/50
4/4 [==============================] - 0s 250us/sample - loss: 0.7029 - acc: 0.5000
Epoch 5/50
4/4 [==============================] - 0s 378us/sample - loss: 0.7029 - acc: 0.5000
Epoch 6/50
4/4 [==============================] - 0s 498us/sample - loss: 0.7029 - acc: 0.5000
Epoch 7/50
4/4 [==============================] - 0s 499us/sample - loss: 0.7028 - acc: 0.5000
Epoch 8/50
4/4 [==============================] - 0s 623us/sample - loss: 0.7028 - acc: 0.5000
Epoch 9/50
4/4 [==============================] - 0s 246us/sample - loss: 0.7028 - acc: 0.5000
Epoch 10/50
4/4 [==============================] - 0s 250us/sample - loss: 0.7028 - acc: 0.5000
Epoch 11/50
4/4 [==============================] 

In [86]:
print(model.predict(X, batch_size=4))

[[0.5036403 ]
 [0.43655676]
 [0.5007963 ]
 [0.44448817]]


In [87]:
print("first layer weights: ", model.layers[0].get_weights()[0])
print("first layer bias: ", model.layers[0].get_weights()[1])

first layer weights:  [[-1.0811887  -0.95438975]
 [-1.1384058   1.2165966 ]]
first layer bias:  [0.00242528 0.00236925]


In [88]:
print("second layer weights: ", model.layers[1].get_weights()[0])
print("second layer bias: ", model.layers[1].get_weights()[1])

second layer weights:  [[ 0.50728   ]
 [-0.51259404]]
second layer bias:  [0.01721454]


In [91]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def get_output(x):
    layer0 = model.layers[0]
    #첫 번째 레이어, 첫 번째 노드의 결과값
    layer0_weights, layer0_bias = layer0.get_weights()
    layer0_node0_weights = np.transpose(layer0_weights)[0]
    layer0_node0_bias = layer0_bias[0]
    layer0_node0_output = sigmoid( np.dot( x, layer0_node0_weights ) + layer0_node0_bias)
    
    #첫 번째 레이어, 두 번째 노드의 결괏값
    layer0_node1_weights = np.transpose(layer0_weights)[1]
    layer0_node1_bias = layer0_bias[1]
    layer0_node1_output = sigmoid( np.dot( x, layer0_node0_weights ) + layer0_node0_bias)
    
    #두 번째 레이어 결괏값
    layer1 = model.layers[1]
    layer1_weights, layer1_bias = layer1.get_weights()
    layer1_output = sigmoid( np.dot( [layer0_node0_output, layer0_node1_output],
                                   layer1_weights ) + layer1_bias)
    
    print(layer1_output)

In [93]:
get_output([0,0])

0.5036385093179494


In [94]:
get_output([0,1])

0.5039806442698742


In [95]:
get_output([1,0])

0.5039664553227073


In [96]:
get_output([1,1])

0.5041730531118043


In [97]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [98]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [99]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [100]:
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


In [101]:
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


In [103]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

In [104]:
model = Sequential([
    Flatten(input_shape=(28, 28)), #데이터 차원 변경
    Dense(256, activation='relu'), #첫번쨰 히든 레이어(h1)
    Dense(128, activation='relu'), #두번째 히든 레이어(h2)
    Dropout(0.1),                  #두번째 히든 레이어(h2)에 드롭아웃(10%) 적용
    Dense(10),                     #세번쨰 히든 레이어(logit)
    Activation('softmax')          #softmax layer
])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [105]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


In [106]:
"""
sparse_categirocal_crossentropy:
레이블을 원 핫 인코딩으로 자동으로 변경해 크로스 엔트로피를 측정합니다.
"""

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [107]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=False),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy',
                           save_best_only=True)]

In [108]:
model.fit(x_train, y_train, epochs=300, batch_size=1000, validation_split=0.1,
         callbacks=callbacks)

Train on 54000 samples, validate on 6000 samples
Epoch 1/300
54000/54000 [==============================] - 2s 40us/sample - loss: 0.6614 - acc: 0.8091 - val_loss: 0.2080 - val_acc: 0.9412
Epoch 2/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.2320 - acc: 0.9319 - val_loss: 0.1460 - val_acc: 0.9602
Epoch 3/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.1685 - acc: 0.9511 - val_loss: 0.1157 - val_acc: 0.9700
Epoch 4/300
54000/54000 [==============================] - 2s 30us/sample - loss: 0.1301 - acc: 0.9621 - val_loss: 0.0991 - val_acc: 0.9727
Epoch 5/300
54000/54000 [==============================] - 2s 30us/sample - loss: 0.1078 - acc: 0.9685 - val_loss: 0.0916 - val_acc: 0.9742
Epoch 6/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0878 - acc: 0.9745 - val_loss: 0.0828 - val_acc: 0.9770
Epoch 7/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0754 - acc: 0.9779 - val_lo

54000/54000 [==============================] - 2s 32us/sample - loss: 0.0131 - acc: 0.9969 - val_loss: 0.0658 - val_acc: 0.9827
Epoch 20/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0134 - acc: 0.9967 - val_loss: 0.0636 - val_acc: 0.9823
Epoch 21/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0116 - acc: 0.9974 - val_loss: 0.0657 - val_acc: 0.9830
Epoch 22/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0092 - acc: 0.9980 - val_loss: 0.0642 - val_acc: 0.9842
Epoch 23/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0083 - acc: 0.9984 - val_loss: 0.0677 - val_acc: 0.9827
Epoch 24/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0069 - acc: 0.9989 - val_loss: 0.0664 - val_acc: 0.9815
Epoch 25/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0074 - acc: 0.9984 - val_loss: 0.0723 - val_acc: 0.9818
Epoch 26/300
54000/54000 [

54000/54000 [==============================] - 2s 34us/sample - loss: 0.0021 - acc: 0.9998 - val_loss: 0.0709 - val_acc: 0.9830
Epoch 38/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0020 - acc: 0.9998 - val_loss: 0.0716 - val_acc: 0.9838
Epoch 39/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0018 - acc: 0.9998 - val_loss: 0.0748 - val_acc: 0.9833
Epoch 40/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0028 - acc: 0.9993 - val_loss: 0.0748 - val_acc: 0.9838
Epoch 41/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0024 - acc: 0.9997 - val_loss: 0.0732 - val_acc: 0.9835
Epoch 42/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0031 - acc: 0.9994 - val_loss: 0.0815 - val_acc: 0.9798
Epoch 43/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0034 - acc: 0.9991 - val_loss: 0.0816 - val_acc: 0.9828
Epoch 44/300
54000/54000 [

54000/54000 [==============================] - 2s 32us/sample - loss: 0.0065 - acc: 0.9980 - val_loss: 0.0911 - val_acc: 0.9813
Epoch 56/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0057 - acc: 0.9982 - val_loss: 0.0821 - val_acc: 0.9822
Epoch 57/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0026 - acc: 0.9994 - val_loss: 0.0878 - val_acc: 0.9812
Epoch 58/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0033 - acc: 0.9991 - val_loss: 0.0935 - val_acc: 0.9827
Epoch 59/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0035 - acc: 0.9989 - val_loss: 0.0829 - val_acc: 0.9825
Epoch 60/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0016 - acc: 0.9997 - val_loss: 0.0768 - val_acc: 0.9857
Epoch 61/300
54000/54000 [==============================] - 2s 34us/sample - loss: 0.0010 - acc: 0.9998 - val_loss: 0.0756 - val_acc: 0.9847
Epoch 62/300
54000/54000 [

Epoch 73/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0013 - acc: 0.9996 - val_loss: 0.0894 - val_acc: 0.9828
Epoch 74/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0046 - acc: 0.9984 - val_loss: 0.0973 - val_acc: 0.9817
Epoch 75/300
54000/54000 [==============================] - 2s 30us/sample - loss: 0.0039 - acc: 0.9988 - val_loss: 0.0909 - val_acc: 0.9830
Epoch 76/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0028 - acc: 0.9993 - val_loss: 0.0830 - val_acc: 0.9827
Epoch 77/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0915 - val_acc: 0.9842
Epoch 78/300
54000/54000 [==============================] - 2s 31us/sample - loss: 8.4025e-04 - acc: 0.9999 - val_loss: 0.0856 - val_acc: 0.9840
Epoch 79/300
54000/54000 [==============================] - 2s 32us/sample - loss: 4.6582e-04 - acc: 0.9999 - val_loss: 0.0904 - val_acc: 0.9818
Epoch

...

54000/54000 [==============================] - 2s 35us/sample - loss: 1.1665e-05 - acc: 1.0000 - val_loss: 0.0969 - val_acc: 0.9865
Epoch 217/300
54000/54000 [==============================] - 2s 34us/sample - loss: 1.1052e-05 - acc: 1.0000 - val_loss: 0.0985 - val_acc: 0.9867
Epoch 218/300
54000/54000 [==============================] - 2s 32us/sample - loss: 9.6441e-06 - acc: 1.0000 - val_loss: 0.0987 - val_acc: 0.9863
Epoch 219/300
54000/54000 [==============================] - 2s 34us/sample - loss: 2.2200e-05 - acc: 1.0000 - val_loss: 0.0997 - val_acc: 0.9867
Epoch 220/300
54000/54000 [==============================] - 2s 35us/sample - loss: 1.2298e-05 - acc: 1.0000 - val_loss: 0.0980 - val_acc: 0.9870
Epoch 221/300
54000/54000 [==============================] - 2s 34us/sample - loss: 8.8940e-06 - acc: 1.0000 - val_loss: 0.0992 - val_acc: 0.9868
Epoch 222/300
54000/54000 [==============================] - 2s 38us/sample - loss: 8.8327e-06 - acc: 1.0000 - val_loss: 0.0993 - val_acc:

Epoch 234/300
54000/54000 [==============================] - 2s 31us/sample - loss: 7.4415e-05 - acc: 1.0000 - val_loss: 0.0906 - val_acc: 0.9860
Epoch 235/300
54000/54000 [==============================] - 2s 32us/sample - loss: 6.7426e-05 - acc: 1.0000 - val_loss: 0.0927 - val_acc: 0.9852
Epoch 236/300
54000/54000 [==============================] - 2s 31us/sample - loss: 1.0900e-04 - acc: 1.0000 - val_loss: 0.0904 - val_acc: 0.9857
Epoch 237/300
54000/54000 [==============================] - 2s 31us/sample - loss: 7.1342e-05 - acc: 1.0000 - val_loss: 0.0919 - val_acc: 0.9860
Epoch 238/300
54000/54000 [==============================] - 2s 30us/sample - loss: 4.5893e-05 - acc: 1.0000 - val_loss: 0.0931 - val_acc: 0.9857
Epoch 239/300
54000/54000 [==============================] - 2s 31us/sample - loss: 3.6641e-05 - acc: 1.0000 - val_loss: 0.0948 - val_acc: 0.9852
Epoch 240/300
54000/54000 [==============================] - 2s 31us/sample - loss: 3.8028e-05 - acc: 1.0000 - val_loss: 0.0

Epoch 252/300
54000/54000 [==============================] - 2s 31us/sample - loss: 2.4661e-05 - acc: 1.0000 - val_loss: 0.0941 - val_acc: 0.9850
Epoch 253/300
54000/54000 [==============================] - 2s 31us/sample - loss: 2.6848e-05 - acc: 1.0000 - val_loss: 0.0952 - val_acc: 0.9848
Epoch 254/300
54000/54000 [==============================] - 2s 32us/sample - loss: 3.4148e-05 - acc: 1.0000 - val_loss: 0.0947 - val_acc: 0.9853
Epoch 255/300
54000/54000 [==============================] - 2s 31us/sample - loss: 1.6835e-05 - acc: 1.0000 - val_loss: 0.0953 - val_acc: 0.9852
Epoch 256/300
54000/54000 [==============================] - 2s 31us/sample - loss: 2.6072e-05 - acc: 1.0000 - val_loss: 0.0960 - val_acc: 0.9852
Epoch 257/300
54000/54000 [==============================] - 2s 31us/sample - loss: 4.1091e-05 - acc: 1.0000 - val_loss: 0.0930 - val_acc: 0.9852
Epoch 258/300
54000/54000 [==============================] - 2s 33us/sample - loss: 2.5055e-05 - acc: 1.0000 - val_loss: 0.0

Epoch 270/300
54000/54000 [==============================] - 2s 31us/sample - loss: 6.3737e-04 - acc: 0.9998 - val_loss: 0.0964 - val_acc: 0.9857
Epoch 271/300
54000/54000 [==============================] - 2s 33us/sample - loss: 1.5421e-04 - acc: 1.0000 - val_loss: 0.0967 - val_acc: 0.9855
Epoch 272/300
54000/54000 [==============================] - 2s 32us/sample - loss: 9.6111e-05 - acc: 1.0000 - val_loss: 0.0971 - val_acc: 0.9848
Epoch 273/300
54000/54000 [==============================] - 2s 31us/sample - loss: 7.5939e-05 - acc: 1.0000 - val_loss: 0.0948 - val_acc: 0.9850
Epoch 274/300
54000/54000 [==============================] - 2s 32us/sample - loss: 3.6236e-05 - acc: 1.0000 - val_loss: 0.0967 - val_acc: 0.9853
Epoch 275/300
54000/54000 [==============================] - 2s 31us/sample - loss: 3.2710e-05 - acc: 1.0000 - val_loss: 0.0961 - val_acc: 0.9853
Epoch 276/300
54000/54000 [==============================] - 2s 31us/sample - loss: 3.7477e-05 - acc: 1.0000 - val_loss: 0.0

Epoch 288/300
54000/54000 [==============================] - 2s 31us/sample - loss: 9.8341e-04 - acc: 0.9996 - val_loss: 0.0990 - val_acc: 0.9845
Epoch 289/300
54000/54000 [==============================] - 2s 34us/sample - loss: 2.2068e-04 - acc: 1.0000 - val_loss: 0.1006 - val_acc: 0.9843
Epoch 290/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0017 - acc: 0.9996 - val_loss: 0.1112 - val_acc: 0.9837
Epoch 291/300
54000/54000 [==============================] - 2s 32us/sample - loss: 0.0017 - acc: 0.9996 - val_loss: 0.1007 - val_acc: 0.9833
Epoch 292/300
54000/54000 [==============================] - 2s 31us/sample - loss: 0.0011 - acc: 0.9997 - val_loss: 0.1032 - val_acc: 0.9838
Epoch 293/300
54000/54000 [==============================] - 2s 31us/sample - loss: 3.3242e-04 - acc: 0.9999 - val_loss: 0.1004 - val_acc: 0.9848
Epoch 294/300
54000/54000 [==============================] - 2s 31us/sample - loss: 3.7799e-04 - acc: 0.9999 - val_loss: 0.1075 - val_ac

In [109]:
results = model.evaluate(x_test, y_test, verbose=0)
print('test loss, test acc:', results)

test loss, test acc: [0.12120633712385279, 0.9822]
